In [1]:
!pip install pydub
import numpy as np
from pydub import AudioSegment
import random
import sys
import io
import os
import glob
import IPython
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pylab
import keras
import tensorflow
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os
from pydub import AudioSegment
%matplotlib inline

In [ ]:
_, data = wavfile.read("/content/drive/MyDrive/Colab Notebooks/ARY Digital/Ruha TV par ARY Digital Lagao 1.wav")
print("Time steps in audio recording before spectrogram", data[:,0].shape)
print("Time steps in input after spectrogram", x.shape)

Time steps in audio recording before spectrogram (176400,)
Time steps in input after spectrogram (101, 2203)


In [21]:
# Calculate and plot spectrogram for a wav audio file
def graph_spectrogram(wav_file):
    rate, data = get_wav_info(wav_file)
    nfft = 200 # Length of each window segment
    fs = 8000 # Sampling frequencies
    noverlap = 120 # Overlap between windows
    nchannels = data.ndim
    if nchannels == 1:
        pxx, freqs, bins, im = plt.specgram(data, nfft, fs, noverlap = noverlap)
    elif nchannels == 2:
        pxx, freqs, bins, im = plt.specgram(data[:,0], nfft, fs, noverlap = noverlap)
    return pxx

# Load a wav file
def get_wav_info(wav_file):
    rate, data = wavfile.read(wav_file)
    return rate, data

# Used to standardize volume of audio clip
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

# Load raw audio files for speech synthesis
def load_raw_audio():
    activates = []
    backgrounds = []
    for filename in os.listdir("/content/drive/MyDrive/Colab Notebooks/FYP Model/data/Channel Detection Dataset/Extra Dataset/HUM TV"):
        if filename.endswith("wav"):
            activate = AudioSegment.from_wav("/content/drive/MyDrive/Colab Notebooks/FYP Model/data/Channel Detection Dataset/Extra Dataset/HUM TV/"+filename)[:3999]
            activates.append(activate)
    for filename in os.listdir("/content/drive/MyDrive/Colab Notebooks/channel_background"):
        if filename.endswith("wav"):
            background = AudioSegment.from_wav("/content/drive/MyDrive/Colab Notebooks/channel_background/"+filename)
            backgrounds.append(background)
    return activates, backgrounds

# Load audio segments using pydub 
activates, backgrounds = load_raw_audio()

print("background len: " + str(len(backgrounds[0])))    # Should be 10,000, since it is a 10 sec clip
print("activate[0] len: " + str(len(activates[0])))     # Maybe around 1000, since an "activate" audio clip is usually around 1 sec (but varies a lot)
print("activate[1] len: " + str(len(activates[1])))     # Different "activate" clips can have different lengths 

background len: 4000
activate[0] len: 3264
activate[1] len: 2923


In [4]:
Tx = 2203 # The number of time steps input to the model from the spectrogram
Ty = 548 # The number of time steps in the output of our model

In [5]:
def get_random_time_segment(segment_ms):
    """
    Gets a random time segment of duration segment_ms in a 10,000 ms audio clip.
    
    Arguments:
    segment_ms -- the duration of the audio clip in ms ("ms" stands for "milliseconds")
    
    Returns:
    segment_time -- a tuple of (segment_start, segment_end) in ms
    """
    
    segment_start = np.random.randint(low=0, high=4000-segment_ms)   # Make sure segment doesn't run past the 10sec background 
    segment_end = segment_start + segment_ms - 1
    
    return (segment_start, segment_end)

In [6]:
# GRADED FUNCTION: is_overlapping

def is_overlapping(segment_time, previous_segments):
    """
    Checks if the time of a segment overlaps with the times of existing segments.
    
    Arguments:
    segment_time -- a tuple of (segment_start, segment_end) for the new segment
    previous_segments -- a list of tuples of (segment_start, segment_end) for the existing segments
    
    Returns:
    True if the time segment overlaps with any of the existing segments, False otherwise
    """
    
    segment_start, segment_end = segment_time
    
    ### START CODE HERE ### (≈ 4 line)
    # Step 1: Initialize overlap as a "False" flag. (≈ 1 line)
    overlap = False
    
    # Step 2: loop over the previous_segments start and end times.
    # Compare start/end times and set the flag to True if there is an overlap (≈ 3 lines)
    for previous_start, previous_end in previous_segments:
        if segment_start <= previous_end and segment_end >= previous_start:
            overlap = True
    ### END CODE HERE ###

    return overlap

In [7]:
# GRADED FUNCTION: insert_audio_clip

def insert_audio_clip(background, audio_clip, previous_segments):
    """
    Insert a new audio segment over the background noise at a random time step, ensuring that the 
    audio segment does not overlap with existing segments.
    
    Arguments:
    background -- a 10 second background audio recording.  
    audio_clip -- the audio clip to be inserted/overlaid. 
    previous_segments -- times where audio segments have already been placed
    
    Returns:
    new_background -- the updated background audio
    """
    
    # Get the duration of the audio clip in ms
    segment_ms = len(audio_clip)
    
    ### START CODE HERE ### 
    # Step 1: Use one of the helper functions to pick a random time segment onto which to insert 
    # the new audio clip. (≈ 1 line)
    segment_time = get_random_time_segment(segment_ms)
    
    # Step 2: Check if the new segment_time overlaps with one of the previous_segments. If so, keep 
    # picking new segment_time at random until it doesn't overlap. (≈ 2 lines)
    while is_overlapping(segment_time, previous_segments):
        segment_time = get_random_time_segment(segment_ms)

    # Step 3: Add the new segment_time to the list of previous_segments (≈ 1 line)
    previous_segments.append(segment_time)
    ### END CODE HERE ###
    
    # Step 4: Superpose audio segment and background
    new_background = background.overlay(audio_clip, position = segment_time[0])
    
    return new_background, segment_time

In [8]:
# GRADED FUNCTION: insert_ones

def insert_ones(y, segment_end_ms):
    """
    Update the label vector y. The labels of the 50 output steps strictly after the end of the segment 
    should be set to 1. By strictly we mean that the label of segment_end_y should be 0 while, the
    50 followinf labels should be ones.
    
    
    Arguments:
    y -- numpy array of shape (1, Ty), the labels of the training example
    segment_end_ms -- the end time of the segment in ms
    
    Returns:
    y -- updated labels
    """
    
    # duration of the background (in terms of spectrogram time-steps)
    segment_end_y = int(segment_end_ms * Ty / 4000.0)
    
    # Add 1 to the correct index in the background label (y)
    ### START CODE HERE ### (≈ 3 lines)
    for i in range(segment_end_y + 1, segment_end_y + 51):
        if i < Ty:
            y[0, i] = 1
    ### END CODE HERE ###
    
    return y

In [22]:
# Preprocess the audio to the correct format
def preprocess_audio(filename):
    # Trim or pad audio segment to 4000ms
    padding = AudioSegment.silent(duration=4000)
    segment = AudioSegment.from_wav(filename)[:4000]
    segment = padding.overlay(segment)
    # Set frame rate to 44100
    segment = segment.set_frame_rate(44100)
    # Export as wav
    segment.export(filename, format='wav')

# GRADED FUNCTION: create_training_example
def create_training_example(background, activate, count):
    """
    Creates a training example with a given background, activates, and negatives.
    
    Arguments:
    background -- a 10 second background audio recording
    activates -- a list of audio segments of the word "activate"
    negatives -- a list of audio segments of random words that are not "activate"
    
    Returns:
    x -- the spectrogram of the training example
    y -- the label at each time step of the spectrogram
    """
    
    # Make background quieter
    background = background - 20

    ### START CODE HERE ###
    # Step 1: Initialize y (label vector) of zeros (≈ 1 line)
    y = np.zeros((1, Ty))

    # Step 2: Initialize segment times as empty list (≈ 1 line)
    previous_segments = []
    ### END CODE HERE ###
        
    background, segment_time = insert_audio_clip(background, activate, previous_segments)
    # Retrieve segment_start and segment_end from segment_time
    segment_start, segment_end = segment_time
    # Insert labels in "y"
    y = insert_ones(y, segment_end_ms=segment_end)
 
    # Standardize the volume of the audio clip 
    background = match_target_amplitude(background, -20.0)

    # Export new training example 
    file_handle = background.export("/content/drive/MyDrive/Colab Notebooks/HUM TV/Ruha TV Par HUM TV Lagao " + str(count) + ".wav", format="wav")
    preprocess_audio("/content/drive/MyDrive/Colab Notebooks/HUM TV/Ruha TV Par HUM TV Lagao " + str(count) + ".wav")
    count += 1
    print("File was saved in your directory.")
    
    # Get and plot spectrogram of the new recording (background with superposition of positive and negatives)
    # x = graph_spectrogram("train.wav")
    
    # return x, y

In [23]:
print(len(activates))
count = 1
for activate in activates:
  for background in backgrounds:
    create_training_example(background, activate, count)
    count += 1


# X = []
# Y = []
# for i in range(1, 201):
#   background_index = np.random.randint(0, len(backgrounds))
#   x, y = create_training_example(backgrounds[0], activates, negatives)
  
#   # If maximum length exceeds mfcc lengths then pad the remaining ones
#   if (x.shape[1] < Tx):
#     pad_width = Tx - x.shape[1]
#     x = np.pad(x, pad_width=((0, 0), (0, pad_width)), mode='constant')
#   # Else cutoff the remaining parts
#   elif (x.shape[1] > Tx):
#     x = x[:, :Tx]
#   X.append(x)
#   Y.append(y)
# X = np.array(X)
# Y = np.array(Y)

92
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was saved in your directory.
File was sa

In [ ]:
print(X.shape)
X = X.reshape(X.shape[0], X.shape[2], X.shape[1])
print(X.shape)
Y = Y.reshape(Y.shape[0], Y.shape[2],  Y.shape[1])
print(Y.shape)

(200, 101, 2203)
(200, 2203, 101)
(200, 548, 1)


Now you can listen to the training example you created and compare it to the spectrogram generated above.

In [ ]:
IPython.display.Audio("train.wav")